In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ObjectDetectionCNN(nn.Module):
    def __init__(self):
        super(ObjectDetectionCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 26 * 19, 256)  # Assuming input image size of 32x32
        self.fc2 = nn.Linear(256, 1)
        
        # Activation functions
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        
        return x


In [5]:
from torch import optim
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, target1_dir, target0_dir, transform=None):
        """
        target1_dir: directory con le immagini per target=1
        target0_dir: directory con le immagini per target=0
        """
        self.target1_images = [os.path.join(target1_dir, f) for f in os.listdir(target1_dir)]
        self.target0_images = [os.path.join(target0_dir, f) for f in os.listdir(target0_dir)]
        self.images = self.target1_images + self.target0_images
        self.labels = [1] * len(self.target1_images) + [0] * len(self.target0_images)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert("RGB")  # Carica l'immagine e la converte in RGB
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Definisci le trasformazioni
transform = transforms.Compose([
    transforms.ToTensor(),  # Converte l'immagine in tensore
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizzazione
])

# Cartelle che contengono le immagini
target1_dir = './data/target1/'  # Sostituisci con il percorso della cartella target1
target0_dir = './data/target0/'  # Sostituisci con il percorso della cartella target0

# Creare il dataset personalizzato
dataset = CustomDataset(target1_dir, target0_dir, transform=transform)

# Separare il dataset in addestramento (80%) e validazione (20%)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Creare DataLoader per il training e la validazione
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Inizializzare il modello
model = ObjectDetectionCNN()

# Impostare la funzione di perdita e l'ottimizzatore
criterion = torch.nn.BCELoss()  # Binary Cross Entropy per un problema di classificazione binaria
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Funzione di addestramento
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())  # Etichette in formato float per BCELoss
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoca {epoch+1}, Loss Addestramento: {running_loss / len(train_loader)}')

        # Valutazione del modello sui dati di validazione
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                predicted = (outputs.squeeze() > 0.5).float()  # Usa la soglia 0.5 per classificare
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Accuratezza Validazione: {accuracy * 100:.2f}%')

# Addestrare il modello
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)

Epoca 1, Loss Addestramento: 0.6874504089355469
Accuratezza Validazione: 83.33%
Epoca 2, Loss Addestramento: 0.6425769925117493
Accuratezza Validazione: 16.67%
Epoca 3, Loss Addestramento: 0.7270681262016296
Accuratezza Validazione: 100.00%
Epoca 4, Loss Addestramento: 0.34211012721061707
Accuratezza Validazione: 100.00%
Epoca 5, Loss Addestramento: 0.14339560270309448
Accuratezza Validazione: 100.00%
